In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
from crewai import LLM

llm =  LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.1
)
llm.call("where is the capital of pakistan")

Output()

'The capital of Pakistan is **Islamabad**.\n'

In [15]:
original_email = """Subject: Status Update – Q3 Initiatives (P1: STR, P2: LRP Alignment)

Dear Team,

I hope this message finds you well.

This is a quick update on our Q3 deliverables. As per the last EXCOM review, we’re on track with both Priority 1 (STR rollout) and Priority 2 (LRP alignment).

Key highlights:

STR pilot completed in Region A – results show a 15% efficiency gain
LRP integration with CRM is 80% complete; UAT scheduled for next week
RACI for Phase 3 has been approved by PMO
Budget burn rate is at 45% – within TTB guidelines
Action Items:

Please submit your WFR by EOD Thursday
Attend the VTM on Friday – agenda to follow
Flag any roadblocks using the BRD portal
Let’s keep the momentum going. Reach out if you need support from the COE.

Best regards,
Nina Patel
Senior Project Manager
Strategic Operations, AlphaTech Solutions
📧 nina.patel@alphatech.com | 📱 (555) 123-4567"""

from crewai.tools import BaseTool
class ReplaceJargonTool(BaseTool):
    name:str = "Replace Jargons"
    description:str ="Replace jargons in the email with simpler and specific terms"
    
    def _run(self,email:str) ->str:
        replacements = {
  "STR": "System Transformation Roadmap",
  "LRP": "Long-Range Planning",
  "EXCOM": "Executive Committee",
  "RACI": "Responsible, Accountable, Consulted, Informed",
  "PMO": "Project Management Office",
  "TTB": "Time-to-Budget (internal performance metric) (fictional)",
  "WFR": "Weekly Feedback Report (or: Weekly Field Report)",
  "EOD": "End of Day",
  "VTM": "Virtual Team Meeting",
  "BRD": "Business Requirements Document",
  "COE": "Center of Excellence",
  "CRM": "Customer Relationship Management",
  "UAT": "User Acceptance Testing"
}
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append((jargon, replacement))
                email = email.replace(jargon, replacement)

        return f"Email after replacing jargons:\n{email}\n\nSuggestions:\n{suggestions}"
    
jt =  ReplaceJargonTool()
jt.run(original_email)  

Using Tool: Replace Jargons

"Email after replacing jargons:\nSubject: Status Update – Q3 Initiatives (P1: System Transformation Roadmap, P2: Long-Range Planning Alignment)\n\nDear Team,\n\nI hope this message finds you well.\n\nThis is a quick update on our Q3 deliverables. As per the last Executive Committee review, we’re on track with both Priority 1 (System Transformation Roadmap rollout) and Priority 2 (Long-Range Planning alignment).\n\nKey highlights:\n\nSystem Transformation Roadmap pilot completed in Region A – results show a 15% efficiency gain\nLong-Range Planning integration with Customer Relationship Management is 80% complete; User Acceptance Testing scheduled for next week\nResponsible, Accountable, Consulted, Informed for Phase 3 has been approved by Project Management Office\nBudget burn rate is at 45% – within Time-to-Budget (internal performance metric) (fictional) guidelines\nAction Items:\n\nPlease submit your Weekly Feedback Report (or: Weekly Field Report) by End of Day Thursday\nAttend the 

In [16]:
from crewai import Agent, Crew,Task

email_assistant = Agent(
    role="Email Assistant Agent",
    goal="Efficiently manage, prioritize, and respond to incoming emails by drafting clear, professional, and context-aware replies based on user intent and priority.",
    backstory="""You are an intelligent and highly organized Email Assistant Agent designed to help users manage their inbox effectively. 
    With a keen eye for detail and a deep understanding of communication tone and context, you analyze incoming emails, determine urgency, 
    and draft appropriate responses. Whether it’s scheduling meetings, answering inquiries, or flagging important messages, 
    you ensure the user stays on top of their communication with minimal effort. 
    You’ve assisted busy professionals across industries, saving them hours each week by automating routine email tasks while maintaining a personalized touch.""",
    verbose=True,
    tools=[jt],
    llm=llm
)


In [17]:

email_task = Task(
    description=f""" 
    Analyze the following email and expand all abbreviations and acronyms into their full, formal forms 
    to improve clarity and professionalism. Ensure the expanded version retains the original meaning and tone.

    Original Email:
    '''{original_email}'''

    Steps:
    1. Identify all common abbreviations (e.g., ASAP → As Soon As Possible, FYI → For Your Information).
    2. Replace each abbreviation with its full form.
    3. Preserve the structure and intent of the original message.
    4. Return the fully expanded, easy-to-understand version of the email.
    """,
    agent=email_assistant,
    expected_output="A professionally rewritten version of the original email with all abbreviations and acronyms expanded into their full forms, while maintaining the original meaning, tone, and structure."
)



In [18]:


crew = Crew(
    agents=[email_assistant],
    tasks=[email_task],
    name="Email Management Crew",
    description="A specialized crew focused on efficiently managing and responding to emails, ensuring clarity and professionalism in communication.",
    verbose=True

)
result =  crew.kickoff()
print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: Email Management Crew                                                                                    │
│  ID: ceddd533-6764-42cc-9764-15d817caa11b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze the following email and expand all abbreviations and acronyms into their full, formal forms        │
│      to improve clarity and professionalism. Ensure the expanded version retains the original meaning and       │
│  tone.                                                                                                          │
│                                                                                                                 │
│      Original Email:                                                                                            │
│      '''Subject: Status Update – Q3 Initiatives (P1: STR, P2: LRP Alignment)                                    │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  I hope this message finds you well.                                                                            │
│                                                                                                                 │
│  This is a quick update on our Q3 deliverables. As per the last EXCOM review, we’re on track with both          │
│  Priority 1 (STR rollout) and Priority 2 (LRP alignment).                                                       │
│                                                                                                                 │
│  Key highlights:                                                                                                │
│                                                                                                                 │
│  STR pilot completed in Region A – results show a 15% efficiency gain                                           │
│  LRP integration with CRM is 80% complete; UAT scheduled for next week                                          │
│  RACI for Phase 3 has been approved by PMO                                                                      │
│  Budget burn rate is at 45% – within TTB guidelines                                                             │
│  Action Items:                                                                                                  │
│                                                                                                                 │
│  Please submit your WFR by EOD Thursday                                                                         │
│  Attend the VTM on Friday – agenda to follow                                                                    │
│  Flag any roadblocks using the BRD portal                                                                       │
│  Let’s keep the momentum going. Reach out if you need support from the COE.                                     │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  Nina Patel                                                                                                     │
│  Senior Project Manager                                

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Thought: I need to expand all abbreviations and acronyms in the email to their full forms to improve clarity   │
│  and professionalism. I will use the Replace Jargons tool to achieve this.                                      │
│                                                                                                                 │
│  Using Tool: Replace Jargons                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"email\": \"Subject: Status Update \\u2013 Q3 Initiatives (P1: STR, P2: LRP Alignment)\\n\\nDear Team,\\n\  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Status Update – Q3 Initiatives (P1: System Transformation Roadmap, P2: Long-Range Planning            │
│  Alignment)                                                                                                     │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  I hope this message finds you well.                                                                            │
│                                                                                                                 │
│  This is a quick update on our Q3 deliverables. As per the last Executive Committee review, we’re on track      │
│  with both Priority 1 (System Transformation Roadmap rollout) and Priority 2 (Long-Range Planning alignment).   │
│                                                                                                                 │
│  Key highlights:                                                                                                │
│                                                                                                                 │
│  System Transformation Roadmap pilot completed in Region A – results show a 15% efficiency gain                 │
│  Long-Range Planning integration with Customer Relationship Management is 80% complete; User Acceptance         │
│  Testing scheduled for next week                                                                                │
│  Responsible, Accountable, Consulted, Informed for Phase 3 has been approved by Project Management Office       │
│  Budget burn rate is at 45% – within Time-to-Budget (internal performance metric) (fictional) guidelines        │
│  Action Items:                                                                                                  │
│                                                                                                                 │
│  Please submit your Weekly Feedback Report (or: Weekly Field Report) by End of Day Thursday                     │
│  Attend the Virtual Team Meeting on Friday – agenda to follow                                                   │
│  Flag any roadblocks using the Business Requirements Document portal                                            │
│  Let’s keep the momentum going. Reach out if you need support from the Center of Excellence.                    │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  Nina Patel                                                                                                     │
│  Senior Project Manager                                                                                         │
│  Strategic Operations, AlphaTech Solutions                                                                      │
│  📧 nina.patel@alphatech.com | 📱 (555) 123-4567                                                                │
│                                                          

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3bf59393-6b53-4436-9fa4-f2f5b4923cb1                                                                     │
│  Agent: Email Assistant Agent                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: Email Management Crew                                                                                    │
│  ID: ceddd533-6764-42cc-9764-15d817caa11b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Status Update – Q3 Initiatives (P1: System Transformation Roadmap, P2: Long-Range       │
│  Planning Alignment)                                                                                            │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  I hope this message finds you well.                                                                            │
│                                                                                                                 │
│  This is a quick update on our Q3 deliverables. As per the last Executive Committee review, we’re on track      │
│  with both Priority 1 (System Transformation Roadmap rollout) and Priority 2 (Long-Range Planning alignment).   │
│                                                                                                                 │
│  Key highlights:                                                                                                │
│                                                                                                                 │
│  System Transformation Roadmap pilot completed in Region A – results show a 15% efficiency gain                 │
│  Long-Range Planning integration with Customer Relationship Management is 80% complete; User Acceptance         │
│  Testing scheduled for next week                                                                                │
│  Responsible, Accountable, Consulted, Informed for Phase 3 has been approved by Project Management Office       │
│  Budget burn rate is at 45% – within Time-to-Budget (internal performance metric) (fictional) guidelines        │
│  Action Items:                                                                                                  │
│                                                                                                                 │
│  Please submit your Weekly Feedback Report (or: Weekly Field Report) by End of Day Thursday                     │
│  Attend the Virtual Team Meeting on Friday – agenda to follow                                                   │
│  Flag any roadblocks using the Business Requirements Document portal                                            │
│  Let’s keep the momentum going. Reach out if you need support from the Center of Excellence.                    │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  Nina Patel                                                                                                     │
│  Senior Project Manager                                                                                         │
│  Strategic Operations, AlphaTech Solutions                                                                      │
│  📧 nina.patel@alphatech.com | 📱 (555) 123-4567        

Subject: Status Update – Q3 Initiatives (P1: System Transformation Roadmap, P2: Long-Range Planning Alignment)

Dear Team,

I hope this message finds you well.

This is a quick update on our Q3 deliverables. As per the last Executive Committee review, we’re on track with both Priority 1 (System Transformation Roadmap rollout) and Priority 2 (Long-Range Planning alignment).

Key highlights:

System Transformation Roadmap pilot completed in Region A – results show a 15% efficiency gain
Long-Range Planning integration with Customer Relationship Management is 80% complete; User Acceptance Testing scheduled for next week
Responsible, Accountable, Consulted, Informed for Phase 3 has been approved by Project Management Office
Budget burn rate is at 45% – within Time-to-Budget (internal performance metric) (fictional) guidelines
Action Items:

Please submit your Weekly Feedback Report (or: Weekly Field Report) by End of Day Thursday
Attend the Virtual Team Meeting on Friday – agenda to follow
